In [20]:
import pandas as pd
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')
print(train_df.head())

import numpy as np
import cv2
from itertools import combinations
from scipy.ndimage import laplace, sobel
from scipy.stats import kurtosis, skew

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [21]:
def get_img_feat(img_org):
    img = img_org.ravel()
    feats = [np.mean(img),np.std(img),np.median(img),np.max(img),np.min(img)]
    return feats

    
def get_other_feat(df):
    band1,band2,band3,angs = [],[],[],[]
    for i, row in df.iterrows():
        tmp_feat = []
        img1 = np.array(row['band_1']).astype('float32')
        img2 = np.array(row['band_2']).astype('float32')
        
        if row['inc_angle'] == 'na':
            ang = -1
        else:
            ang = float(row['inc_angle'])
            

        img3 = (img1-img2)*ang/2.0
        band1.append(img1)
        band2.append(img2)
        band3.append(img3)
        tmp_feat = [ang] + get_img_feat(img1) + get_img_feat(img2)
        angs.append(tmp_feat)
    return band1,band2,band3,angs
        
        

a_band1,a_band2,a_band3,a_angs = get_other_feat(train_df)
a_bands = np.hstack([a_band1,a_band2,a_band3])
b_band1,b_band2,b_band3,b_angs = get_other_feat(test_df)
b_bands = np.hstack([b_band1,b_band2,b_band3])
print('raw feats')

raw feats


In [28]:
from sklearn import decomposition
pca_b1 = decomposition.PCA(n_components=20, whiten=True, random_state=15)
a_band_feat = pca_b1.fit_transform(np.array(a_bands))
b_band_feat = pca_b1.transform(np.array(b_bands))
print('pca done',a_band_feat.shape)

pca done (1604, 20)


In [29]:
train_feat = np.hstack([a_band_feat,np.array(a_angs)])
test_feat = np.hstack([b_band_feat,np.array(b_angs)])

print(train_feat.shape,test_feat.shape)

(1604, 31) (8424, 31)


In [30]:
import pickle
with open('../features/other_feat.pkl','wb') as fout:
    pickle.dump([train_feat,test_feat],fout)
print('done')

done
